# Parameter Exploration - Assessing effects of FluxRETAP parameters on number of returned targets.

FluxRETAP offers several adjustable parameters that directly influence the quantity of targets returned. A detailed description of these parameters can be found in the [FluxRETAP_Tutorial notebook](http://localhost:8890/lab/tree/FluxRETAP/jupyter%20notebooks/FluxRETAP_Tutorial.ipynb). 

This notebook serves as a guide for understanding how these parameters empirically effect the number of returned targets. In this notebook, we observe that the two parameters, **fluxRangeDiff** and **referenceCutOff**, have the most significant impact on the number of returned reactions across three genome-scale models. In contrast, parameters such as step size (**N**) and minimal biomass growth allowed (**optimalFraction**) have more limited effects.

## Test Case Description.


Here, the FluxRETAP parameters are systematically varied across a spectrum of values in three distinct Genome-Scale Models (GSMs): *E. coli* iJO1366, *P. putida* iJN1463, and *S. cerevisiae* iMM904. The target molecule for each simulation is the heterologous product, indigoidine. Adding indogidine production to each GSM necessitates the inclusion of two reactions, APNPT (creating the indigoidine precursor) and the indigoidine production reaction. Towards the conclusion of the notebook, we compare the total count of returned reactions and the subset of returned reactions that have associated genes for each simulated case. We also tally the total number of active reactions (reactions carrying flux) to evalulate what percent of active reactions are returned as targets. 

# Setup

## Warning! 
This notebook takes ~2 hours to run on an Apple M2 Pro Notebook (36 FluxRETAP runs, 144 FVA simulations). 


The first step to use this notebook is to make sure that the required dependencies are installed.
* cobra
* scipy
* pandas
* numpy
* matplotlib

if the packages are not installed, you can use the following line from the command prompt to install:
> pip install cobra scipy pandas numpy matplotlib

## imports

Imports for basic functionality:

In [1]:
import cobra
from cobra import Metabolite, Reaction
from scipy import stats
import numpy as np
from scipy.stats import norm
import math 
import timeit
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from cobra.flux_analysis import flux_variability_analysis as fva
%matplotlib inline

from IPython.display import Image   # Used to display images

And here we load the FluxRETAP library, make sure you have the right path in libraryPath:

In [2]:
libraryPath = "../../FluxRETAP"
sys.path.append(libraryPath)

from core.FluxRETAP import FluxRETAP

You are using cobra version:  0.29.0


# *E. coli* simulations

## GSM preparation

* load the GSM (*E. coli* model [iJO1366](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3261703/) modified to include the MVA pathway)
* add the heterogenous indigoidine pathway

for instructions on using cobra, visit the [COBRApy documentation](https://opencobra.github.io/cobrapy/)


In [3]:
## set the path to the GSM to be used
# E. coli GSM iJO1366
file_name = '../models/iJO1366_MVA.json' 

# Load the GSM (json format)
ecoli = cobra.io.load_json_model(file_name)

model = ecoli.copy()

### Add heterologous indigoidine production pathway 

Two reactions are needed to enable indigoidine production:
1. APNPT (creating indigoidine precursor)
2. Indigoidine production reaction

The next 5 cells below add the pathway and metabolites. 

In [4]:
# initialize the reaction object and set the bounds (APNPT)
APNPT = Reaction('APNPT')
APNPT.name = 'ATP:pantetheine 4\'-phosphotransferase'
APNPT.subsystem = 'Cytosol'
APNPT.lower_bound = -1000
APNPT.upper_bound = 1000


# initialize the reaction object and set the bounds (Indigoidine production)
product_rxn = Reaction('indigoidine_production')
product_rxn.name = 'Indigoidine production'
product_rxn.subsystem = 'Cytosol'
product_rxn.lower_bound = 0
product_rxn.upper_bound = 1000

# add reaction to the model
model.add_reactions([product_rxn,APNPT])

In [5]:
# initialize non-native metabolites 

# metabolite one (Pantetheine)
ptth_c = Metabolite(
    'ptth_c',
    formula='C11H22N2O4S',
    name='Pantetheine',
    compartment='c'
)


# metabolite two (Indigoidine)
indigoidine_c = Metabolite(
    'indigoidine_c',
    formula ='C10H8N4O4',
    name = 'Indigoidine',
    compartment='c'
)

In [6]:
### add the stochiometrically balanced reaction to the model
## note - the names are metabolites names as they appear in the GSM
# APNPT reaction
APNPT.add_metabolites({
    'atp_c': -1,
    ptth_c: -1,
    'adp_c': 1,
    'h_c':1,
    'pan4p_c':1
})

# Indigodine product reaction
product_rxn.add_metabolites({
    'gln__L_c': -2,
    'atp_c': -2,
    'coa_c': -2,
    'fmn_c': -2,
    'o2_c': -2.5,
    'pap_c':2,
    'fmnh2_c': 2,
    'ppi_c': 2,
    'amp_c': 2,
    ptth_c: 2,
    'h2o_c': 1,
    'pi_c': 2,
    indigoidine_c: 1   
})


Verify that the reactions have been added and that the mass is balanced. 

In [7]:
# APNPT reaction
model.reactions.get_by_id('APNPT').check_mass_balance()

{}

In [8]:
# Indigoidine product reaction
model.reactions.indigoidine_production.check_mass_balance()

{'charge': -6.0}

In [9]:
# Create a demand reaction for indigoidine to allow it to accumulate in the model
demand = model.add_boundary(model.metabolites.indigoidine_c,type="demand")

## Simulations

In [10]:
# start time for evalulating lenght of simulations
start_time = timeit.default_timer()

### Constant parameters

parameters that are constant across all *E. coli* simulations. 

In [11]:
# biomass equation
biomassRxn = 'BIOMASS_Ec_iJO1366_core_53p95M'

# substrate equation
uptakeRxn = 'EX_glc__D_e'

# product reaction
productRxn = 'indigoidine_production'

# reactions up and down regulated
UpOrDown = 'Both'

# return extra data
returnGaussian=False 

In [12]:
# subsystems to be utilized 
subsystems=[]
for r in model.reactions:
    subsystems.append(model.reactions.get_by_id(r.id).subsystem)

desired_systems = set(subsystems)

# remove demand, biomass, exchange reactions
delete = ['Intracellular demand','Extracellular exchange','Biomass and maintenance functions','Inorganic Ion Transport and Metabolism']
desired_systems = desired_systems - set(delete)
desired_systems

{'',
 'Alanine and Aspartate Metabolism',
 'Alternate Carbon Metabolism',
 'Anaplerotic Reactions',
 'Arginine and Proline Metabolism',
 'Cell Envelope Biosynthesis',
 'Citric Acid Cycle',
 'Cofactor and Prosthetic Group Biosynthesis',
 'Cysteine Metabolism',
 'Cytosol',
 'Folate Metabolism',
 'Glutamate Metabolism',
 'Glycerophospholipid Metabolism',
 'Glycine and Serine Metabolism',
 'Glycolysis/Gluconeogenesis',
 'Glyoxylate Metabolism',
 'Histidine Metabolism',
 'Lipopolysaccharide Biosynthesis / Recycling',
 'Membrane Lipid Metabolism',
 'Methionine Metabolism',
 'Methylglyoxal Metabolism',
 'Murein Biosynthesis',
 'Murein Recycling',
 'Nitrogen Metabolism',
 'Nucleotide Salvage Pathway',
 'Oxidative Phosphorylation',
 'Pentose Phosphate Pathway',
 'Purine and Pyrimidine Biosynthesis',
 'Pyruvate Metabolism',
 'S_Sterol_Metabolism',
 'Threonine and Lysine Metabolism',
 'Transport, Inner Membrane',
 'Transport, Outer Membrane',
 'Transport, Outer Membrane Porin',
 'Tyrosine, Trypto

### Simulation a - no cutoffs

In [13]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)



Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.374528301886791

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 236.48309898376465 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 6612074258
##################



In [14]:
# store the number of returned reactions and returned reactions with associated genes.
a = len(indigoidine)
a1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  942
reactions returned with genes:  869


### Simulation b - biomass 50% 

In [15]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.5
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 2.193092078411279

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 172.2629361152649 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 471 and the average is 5674431110
##################



In [16]:
# store the number of returned reactions and returned reactions with associated genes.

b = len(indigoidine)
b1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  945
reactions returned with genes:  872


### Simulation c - biomass 10%

In [17]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.1
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.9402919823724334

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 164.72560501098633 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 446 and the average is 6175266002
##################



In [18]:
# store the number of returned reactions and returned reactions with associated genes.

c= len(indigoidine)
c1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  946
reactions returned with genes:  873


### Simulation d - biomass 90%

In [19]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.9
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.43867685225147124

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 137.16003012657166 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 547 and the average is 1401904137
##################



In [20]:
# store the number of returned reactions and returned reactions with associated genes.

d = len(indigoidine)
d1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  942
reactions returned with genes:  869




### Simulation e - fluxRangeDiff = 0.01 $\frac{mmol}{gDCW.h} $

In [21]:
referenceCutOff = 0
fluxRangeDiff=0.01
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.374528301886773

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 150.40874123573303 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 6612074268
##################



In [22]:
# store the number of returned reactions and returned reactions with associated genes.

e = len(indigoidine)
e1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  320
reactions returned with genes:  301


### Simulation f - fluxRangeDiff $\frac{mmol}{gDCW.h} $

In [23]:
referenceCutOff = 0
fluxRangeDiff=0.05
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.3745283018867855

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 142.85184907913208 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 6612074263
##################



In [24]:
# store the number of returned reactions and returned reactions with associated genes.

f = len(indigoidine)
f1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))


# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  66
reactions returned with genes:  61


### Simulation g - fluxRangeDiff = 0.1 $\frac{mmol}{gDCW.h} $

In [25]:
referenceCutOff = 0
fluxRangeDiff=0.1
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.374528301886782

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 141.8318018913269 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 6612074264
##################



In [26]:
# store the number of returned reactions and returned reactions with associated genes.

g = len(indigoidine)
g1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  50
reactions returned with genes:  46


### Simulation h - referenceCutOff = 500

In [27]:
referenceCutOff = 500
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.374528301886785

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 141.25424790382385 seconds ---

Calculating significance scores
##################
Your cuttoff score is 500
The median reaction score in the dataset is 449 and the average is 6612074261
##################



In [28]:
# store the number of returned reactions and returned reactions with associated genes.

h = len(indigoidine)
h1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  112
reactions returned with genes:  107


### Simulation j - referenceCutOff = 1500
no simulation 'i', 'i' is used as a variable used in a for loop

In [29]:
referenceCutOff = 1500
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.3745283018868095

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 141.7411081790924 seconds ---

Calculating significance scores
##################
Your cuttoff score is 1500
The median reaction score in the dataset is 449 and the average is 6612074262
##################



In [30]:
# store the number of returned reactions and returned reactions with associated genes.

j = len(indigoidine)
j1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))


# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  56
reactions returned with genes:  54


### Simulation k - referenceCutOff = 5000

In [31]:
referenceCutOff = 5000
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 4.374528301886768

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 147.36258506774902 seconds ---

Calculating significance scores
##################
Your cuttoff score is 5000
The median reaction score in the dataset is 449 and the average is 6612074266
##################



In [32]:
# store the number of returned reactions and returned reactions with associated genes.

k = len(indigoidine)
k1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  41
reactions returned with genes:  38


### Simulation m - N=100

In [33]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=100
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.01, 0.99, 1.0]
Maximum flux to product: 4.37452830188678

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 140.95221185684204 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 457 and the average is inf
##################



In [34]:

# store the number of returned reactions and returned reactions with associated genes.
m = len(indigoidine)
m1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  944
reactions returned with genes:  871


### Simulation n - N = 100

In [35]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=1000
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.001, 0.999, 1.0]
Maximum flux to product: 4.374528301886783

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 138.99602103233337 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 458 and the average is inf
##################



In [36]:
# store the number of returned reactions and returned reactions with associated genes.
n = len(indigoidine)
n1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  948
reactions returned with genes:  875


### Save the *E. coli* results. 

saved in a dataframe for analysis at completion of all simulations. 

In [37]:
# temporary store the results. 
results = [[a,a1],
           [b,b1],
           [c,c1],
           [d,d1],
           [e,e1],
           [f,f1],
           [g,g1],
           [h,h1],
           [j,j1],
           [k,k1],
           [m,m1],
           [n,n1]]

In [38]:
results_ecoli = results.copy()
results_ecoli

[[942, 869],
 [945, 872],
 [946, 873],
 [942, 869],
 [320, 301],
 [66, 61],
 [50, 46],
 [112, 107],
 [56, 54],
 [41, 38],
 [944, 871],
 [948, 875]]

### Simulation time

In [39]:
end_time = timeit.default_timer()
execution_time = end_time - start_time
minutes = int(execution_time // 60)
seconds = int(execution_time % 60)
print("Execution time:", minutes, "minutes and", seconds, "seconds")

Execution time: 35 minutes and 6 seconds


# *P. putida* simulations

## GSM preparation

* load the GSM (*P. putida* model [iJN1463]()
* add the heterogenous indigoidine pathway

for instructions on using cobra, visit the [COBRApy documentation](https://opencobra.github.io/cobrapy/)

In [40]:
## set the path to the GSM to be used
# P. putida GSM iJN1463
file_name = '../models/iJN1463_modified.json' 


# load the file (json format)
putida = cobra.io.load_json_model(file_name)

### Add heterologous indigoidine production pathway 

Two reactions are needed to enable indigoidine production:
1. APNPT (creating indigoidine precursor)
2. Indigoidine production reaction

The next 5 cells below add the pathway and metabolites. 

In [41]:
# initialize the reaction object and set the bounds (APNPT)
APNPT = Reaction('APNPT')
APNPT.name = 'ATP:pantetheine 4\'-phosphotransferase'
APNPT.subsystem = 'Cytosol'
APNPT.lower_bound = -1000
APNPT.upper_bound = 1000


# initialize the reaction object and set the bounds (Indigoidine production)
product_rxn = Reaction('indigoidine_production')
product_rxn.name = 'Indigoidine production'
product_rxn.subsystem = 'Cytosol'
product_rxn.lower_bound = 0
product_rxn.upper_bound = 1000

#vadd reaction to the model
putida.add_reactions([product_rxn,APNPT])


In [42]:
# initialize the non-native metabolites 

# metabolite one (Pantetheine)
ptth_c = Metabolite(
    'ptth_c',
    formula='C11H22N2O4S',
    name='Pantetheine',
    compartment='c'
)


# metabolite two (Indigoidine)
indigoidine_c = Metabolite(
    'indigoidine_c',
    formula ='C10H8N4O4',
    name = 'Indigoidine',
    compartment='c'
)

In [43]:
### add the stochiometrically balanced reaction to the model
## note - the names are metabolites names as they appear in the GSM
# APNPT reaction
APNPT.add_metabolites({
    'atp_c': -1,
    ptth_c: -1,
    'adp_c': 1,
    'h_c':1,
    'pan4p_c':1
})

# Indigodine product reaction
product_rxn.add_metabolites({
    'gln__L_c': -2,
    'atp_c': -2,
    'coa_c': -2,
    'fmn_c': -2,
    'o2_c': -2.5,
    'pap_c':2,
    'fmnh2_c': 2,
    'ppi_c': 2,
    'amp_c': 2,
    ptth_c: 2,
    'h2o_c': 1,
    'pi_c': 2,
    indigoidine_c: 1   
})


Verify that the reactions have been added and that the mass is balanced. 

In [44]:
# APNPT reaction
putida.reactions.get_by_id('APNPT').check_mass_balance()

{}

In [45]:
# Indigoidine production reaction
putida.reactions.indigoidine_production.check_mass_balance()

{'charge': -6.0}

In [46]:
# Create a demand reaction for indigoidine to allow it to accumulate in the model
demand = putida.add_boundary(putida.metabolites.indigoidine_c,type="demand")

In [47]:
model = putida.copy()

## Simulations

### Constant parameters

parameters that are constant across all *P. putida* simulations. 

In [48]:
# biomass equation.
biomassRxn = 'BIOMASS_KT2440_WT3'

# product reaction
productRxn = 'indigoidine_production'

# substrate equation
uptakeRxn = 'EX_glc__D_e'

# reactions up and down regulated
UpOrDown = 'Both'

# return extra data
returnGaussian=False

In [49]:
# subsystems to be utilized 
putidaSubsystem = set()

for r in putida.reactions:
    putidaSubsystem.add(r.subsystem)

    
delete = ['Intracellular demand','Extracellular exchange','Biomass and maintenance functions','Intracellular source/sink']
putidaSubsystem = putidaSubsystem-set(delete)

In [50]:
desired_systems = putidaSubsystem

### Simulation a - no cutoffs

In [51]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)



Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.099141104294484

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 146.4347438812256 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 140069318564
##################



In [52]:
# store the number of returned reactions and returned reactions with associated genes.
a = len(indigoidine)
a1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  939
reactions returned with genes:  758


### Simulation b - biomass 50%

In [53]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.5
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 1.59975308641975

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 141.8446638584137 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 471 and the average is 9602380343
##################



In [54]:
# store the number of returned reactions and returned reactions with associated genes.
b = len(indigoidine)
b1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  940
reactions returned with genes:  762


### Simulation c - biomass 10%

In [55]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.1
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 2.8328296294400097

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 141.94119119644165 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 445 and the average is 157376164843
##################



In [56]:
# store the number of returned reactions and returned reactions with associated genes.
c= len(indigoidine)
c1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  940
reactions returned with genes:  757


### Simulation d - biomass 90%

In [57]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.9
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.3199506172839459

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 149.1859791278839 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 547 and the average is 2644741165
##################



In [58]:
# store the number of returned reactions and returned reactions with associated genes.
d = len(indigoidine)
d1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  941
reactions returned with genes:  763


### Simulation e - fluxRangeDiff = 0.01 $\frac{mmol}{gDCW.h} $

In [59]:
referenceCutOff = 0
fluxRangeDiff=0.01
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.099141104294473

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 145.5417981147766 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 139671599068
##################



In [60]:
# store the number of returned reactions and returned reactions with associated genes.
e = len(indigoidine)
e1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  364
reactions returned with genes:  308


### Simulation f - fluxRangeDiff =$\frac{mmol}{gDCW.h} $

In [61]:
referenceCutOff = 0

productRxn = 'indigoidine_production'
fluxRangeDiff=0.05
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.0991411042944805

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 146.0603289604187 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 139671599060
##################



In [62]:
# store the number of returned reactions and returned reactions with associated genes.

f = len(indigoidine)
f1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  72
reactions returned with genes:  66


### Simulation g - fluxRangeDiff = 0.1 $\frac{mmol}{gDCW.h} $

In [63]:
referenceCutOff = 0
fluxRangeDiff=0.1
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.0991411042944828

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 146.44316387176514 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 139750957919
##################



In [64]:
# store the number of returned reactions and returned reactions with associated genes.
g = len(indigoidine)
g1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  49
reactions returned with genes:  44


### Simulation h - referenceCutOff = 500

In [65]:
referenceCutOff = 500
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.099141104294491

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 145.8598279953003 seconds ---

Calculating significance scores
##################
Your cuttoff score is 500
The median reaction score in the dataset is 449 and the average is 140069318596
##################



In [66]:
# store the number of returned reactions and returned reactions with associated genes.

h = len(indigoidine)
h1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  150
reactions returned with genes:  140


### Simulation j - referenceCutOff = 1500

no simulation 'i', 'i' is used as a variable used in a for loop

In [67]:
referenceCutOff = 1500
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.0991411042944756

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 157.00131797790527 seconds ---

Calculating significance scores
##################
Your cuttoff score is 1500
The median reaction score in the dataset is 449 and the average is 139909946493
##################



In [68]:
# store the number of returned reactions and returned reactions with associated genes.

j = len(indigoidine)
j1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  50
reactions returned with genes:  45


### Simulation k - referenceCutOff = 5000

In [69]:
referenceCutOff = 5000
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 3.0991411042944845

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 146.13212203979492 seconds ---

Calculating significance scores
##################
Your cuttoff score is 5000
The median reaction score in the dataset is 449 and the average is 140149130141
##################



In [70]:
# store the number of returned reactions and returned reactions with associated genes.
k = len(indigoidine)
k1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  39
reactions returned with genes:  33


### Simulation m - N = 100

In [71]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=100
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.01, 0.99, 1.0]
Maximum flux to product: 3.0991411042944788

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 146.76074695587158 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 457 and the average is inf
##################



In [72]:
# store the number of returned reactions and returned reactions with associated genes.

m = len(indigoidine)
m1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  939
reactions returned with genes:  760


### Simulation n - N = 1000

In [73]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=1000
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.001, 0.999, 1.0]
Maximum flux to product: 3.09914110429449

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 146.13414120674133 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 458 and the average is inf
##################



In [74]:
# store the number of returned reactions and returned reactions with associated genes.
n = len(indigoidine)
n1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  947
reactions returned with genes:  764


### Save the *P. putida* results. 

saved in a dataframe for analysis at completion of all simulations. 

In [75]:
# temporary store the results. 

results = [[a,a1],
           [b,b1],
           [c,c1],
           [d,d1],
           [e,e1],
           [f,f1],
           [g,g1],
           [h,h1],
           [j,j1],
           [k,k1],
           [m,m1],
           [n,n1]]

In [76]:
results_putida = results.copy()
results_putida

[[939, 758],
 [940, 762],
 [940, 757],
 [941, 763],
 [364, 308],
 [72, 66],
 [49, 44],
 [150, 140],
 [50, 45],
 [39, 33],
 [939, 760],
 [947, 764]]

### Simulation time

In [77]:
end_time = timeit.default_timer()
execution_time = end_time - start_time
minutes = int(execution_time // 60)
seconds = int(execution_time % 60)
print("Execution time:", minutes, "minutes and", seconds, "seconds")

Execution time: 68 minutes and 45 seconds


# *S. cerevisiae* simulations

## GSM preparation

* load the GSM (*S. cerevisiae* model [iMM904]()
* add the heterogenous indigoidine pathway

for instructions on using cobra, visit the [COBRApy documentation](https://opencobra.github.io/cobrapy/)

In [78]:
## set the path to the GSM to be used
# S. cerevisiae model iMM904
file_name = '../models/iMM904.json' 


# load the file (json format)
Scerevisiae = cobra.io.load_json_model(file_name)


### Add heterologous indigoidine production pathway 

Two reactions are needed to enable indigoidine production:
1. APNPT (creating indigoidine precursor)
2. Indigoidine production reaction

The next 5 cells below add the pathway and metabolites. 


change Indigoidine reaction - no flux through fmn_c --> removed from reaction 
Following example published [here](https://www.nature.com/articles/s41467-020-19171-4#Sec20) in [supplemental file 4](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-020-19171-4/MediaObjects/41467_2020_19171_MOESM8_ESM.txt)

In [79]:
## reactions have slightly different stoichiometry for S. cerevisiae. 
# Re-initializing reactions. 

# initialize the reaction object and set the bounds (APNPT)
APNPT = Reaction('APNPT')
APNPT.name = 'ATP:pantetheine 4\'-phosphotransferase'
APNPT.subsystem = 'Cytosol'
APNPT.lower_bound = -1000
APNPT.upper_bound = 1000


# initialize the reaction object and set the bounds (Indigoidine production)
product_rxn = Reaction('indigoidine_production')
product_rxn.name = 'Indigoidine production'
product_rxn.subsystem = 'Cytosol'
product_rxn.lower_bound = 0
product_rxn.upper_bound = 1000


In [80]:
# initialize the non-native metabolites 

# metabolite one (Pantetheine)
ptth_c = Metabolite(
    'ptth_c',
    formula='C11H22N2O4S',
    name='Pantetheine',
    compartment='c'
)


# metabolite two (Indigoidine)
indigoidine_c = Metabolite(
    'indigoidine_c',
    formula ='C10H8N4O4',
    name = 'Indigoidine',
    compartment='c'
)


In [81]:
# ### add the stochiometrically balanced reaction to the model
# ## note - the names are metabolites names as they appear in the GSM

# add reactions to the model
Scerevisiae.add_reactions([product_rxn,APNPT])

# APNPT reaction
APNPT.add_metabolites({
    'atp_c': -1,
    ptth_c: -1,
    'adp_c': 1,
    'h_c':1,
    'pan4p_c':1
})

# Indigodine product reaction
product_rxn.add_metabolites({
    'gln__L_c': -2,
    'atp_c': -2,
    'coa_c': -2,
    'o2_c': -2.5,
    'pap_c':2,
    'ppi_c': 2,
    'amp_c': 2,
    ptth_c: 2,
    'h2o_c': 1,
    'pi_c': 2,
    indigoidine_c: 1   
})

Verify that the reactions have been added and that the mass is balanced. 

In [82]:
# APNPT reaction
Scerevisiae.reactions.get_by_id('APNPT').check_mass_balance()


{}

In [83]:
# Indigoidine reaction
Scerevisiae.reactions.indigoidine_production.check_mass_balance()

{'charge': -6.0, 'H': -4}

In [84]:
# Create a demand reaction for indigoidine to allow it to accumulate in the model
demand = Scerevisiae.add_boundary(Scerevisiae.metabolites.indigoidine_c,type="demand")

In [85]:
model = Scerevisiae.copy()

## Simulations

### Constant parameters

parameters that are constant across all *S. cerevisiae* simulations. 

In [86]:
# biomass equation
biomassRxn = 'BIOMASS_SC5_notrace'

# product reaction
productRxn = 'indigoidine_production'

# substrate equation
uptakeRxn = 'EX_glc__D_e'

# reactions up and down regulated
UpOrDown = 'Both'

# return extra data
returnGaussian=False


In [87]:
# subsystems
ScerevisiaeSubsystem = set()
for r in Scerevisiae.reactions:
    ScerevisiaeSubsystem.add(r.subsystem)
delete = ['Intracellular demand','Extracellular exchange','Biomass and maintenance functions','Intracellular source/sink']

ScerevisiaeSubsystem = ScerevisiaeSubsystem-set(delete)

In [88]:
desired_systems = ScerevisiaeSubsystem

### Simulation a - no cutoffs

In [89]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)



Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.7961165048543708

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 118.7335090637207 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 697359320914
##################



In [90]:
# store the number of returned reactions and returned reactions with associated genes.
a = len(indigoidine)
a1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  530
reactions returned with genes:  359


### Simulation b - biomass 50%

In [91]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.5
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.434807887131525

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 118.68306183815002 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 479 and the average is 3597124498
##################



In [92]:
# store the number of returned reactions and returned reactions with associated genes.
b = len(indigoidine)
b1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  536
reactions returned with genes:  364


### Simulation c - biomass 10%

In [93]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.1
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.7445274846876213

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 117.25062894821167 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 448 and the average is 3966947687
##################



In [94]:
# store the number of returned reactions and returned reactions with associated genes.
c= len(indigoidine)
c1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  549
reactions returned with genes:  371


### Simulation d - biomass 90%

In [95]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0.9
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.09974128760219901

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 115.95865607261658 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 548 and the average is 802057937
##################



In [96]:
# store the number of returned reactions and returned reactions with associated genes.
d = len(indigoidine)
d1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  535
reactions returned with genes:  360


### Simulation e - fluxRangeDiff = 0.01 $\frac{mmol}{gDCW.h} $

In [97]:
referenceCutOff = 0
fluxRangeDiff=0.01
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.796116504854376

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 112.91496896743774 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 699663371488
##################



In [98]:
# store the number of returned reactions and returned reactions with associated genes.
e = len(indigoidine)
e1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  83
reactions returned with genes:  55


### Simulation f - fluxRangeDiff = 0.05 $\frac{mmol}{gDCW.h} $

In [99]:
referenceCutOff = 0
fluxRangeDiff=0.05
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.7961165048543686

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 113.23944902420044 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 698125649688
##################



In [100]:
# store the number of returned reactions and returned reactions with associated genes.
f = len(indigoidine)
f1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  41
reactions returned with genes:  24


### Simulation g - fluxRangeDiff = 0.1 $\frac{mmol}{gDCW.h} $

In [101]:
referenceCutOff = 0
fluxRangeDiff=0.1
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.7961165048543657

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 113.27696990966797 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 449 and the average is 697359320809
##################



In [102]:
# store the number of returned reactions and returned reactions with associated genes.

g = len(indigoidine)
g1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  31
reactions returned with genes:  15


### Simulation h - referenceCutOff = 500

In [103]:
referenceCutOff = 500
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.796116504854351

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 113.21208500862122 seconds ---

Calculating significance scores
##################
Your cuttoff score is 500
The median reaction score in the dataset is 449 and the average is 701982697390
##################



In [104]:
# store the number of returned reactions and returned reactions with associated genes.
h = len(indigoidine)
h1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  138
reactions returned with genes:  104


### Simulation j - referenceCutOff = 1500

no simulation 'i', 'i' is used as a variable used in a for loop

In [105]:
referenceCutOff = 1500
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.7961165048543662

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 111.9221830368042 seconds ---

Calculating significance scores
##################
Your cuttoff score is 1500
The median reaction score in the dataset is 449 and the average is 695831699111
##################



In [106]:
# store the number of returned reactions and returned reactions with associated genes.
j = len(indigoidine)
j1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  50
reactions returned with genes:  41


### Simulation k - referenceCutOff = 500

In [107]:
referenceCutOff = 5000
fluxRangeDiff=0
optimalFraction=0
N=10
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.1, 0.9, 1.0]
Maximum flux to product: 0.7961165048543385

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 120.85955882072449 seconds ---

Calculating significance scores
##################
Your cuttoff score is 5000
The median reaction score in the dataset is 449 and the average is 700434775408
##################



In [108]:
# store the number of returned reactions and returned reactions with associated genes.

k = len(indigoidine)
k1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  42
reactions returned with genes:  33


### Simulation m - N = 100

In [109]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=100
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.01, 0.99, 1.0]
Maximum flux to product: 0.7961165048543586

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 110.07252717018127 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 457 and the average is inf
##################



In [110]:

# store the number of returned reactions and returned reactions with associated genes.
m = len(indigoidine)
m1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  538
reactions returned with genes:  364


### Simulation n - N = 800

setting step to 1000 takes very long time to complete. 

In [111]:
referenceCutOff = 0
fluxRangeDiff=0
optimalFraction=0
N=800 # setting step to 1000 takes very long time to complete. 
indigoidine = FluxRETAP.getRecommendations(model=model,
                                 UpOrDown=UpOrDown,
                                 productRxn=productRxn,
                                 carbonSourceRxn=uptakeRxn,
                                 biomassRxn=biomassRxn,
                                 desiredSystems=desired_systems,
                                 N=N, 
                                 referenceCutOff=referenceCutOff,
                                 
                                 returnGaussian=returnGaussian,
                                 optimalFraction=optimalFraction,
                                 fluxRangeDiff=fluxRangeDiff)


Fractions of flux: [0.0, 0.00125, 0.99875, 1.0]
Maximum flux to product: 0.7961165048543624

starting FVA simulations . . .
25.0 % complete
50.0 % complete
75.0 % complete
100.0 % complete
--- 113.6689522266388 seconds ---

Calculating significance scores
##################
Your cuttoff score is 0
The median reaction score in the dataset is 458 and the average is inf
##################



In [112]:
# store the number of returned reactions and returned reactions with associated genes.
n = len(indigoidine)
n1 = len(indigoidine[indigoidine['bgene']!=""])
print('total reactions returned : ',len(indigoidine))

print('reactions returned with genes: ',len(indigoidine[indigoidine['bgene']!=""]))

# # print number of reactions per subsystem
# subsystems=[]
# for r in indigoidine.index:
#     subsystems.append(model.reactions.get_by_id(r).subsystem)

# for i in set(subsystems):
#     print(i)
#     print((subsystems.count(i)))

total reactions returned :  538
reactions returned with genes:  361


### Save the *S. cerevisiae* results

In [113]:
# temporary store the results. 
results = [[a,a1],
           [b,b1],
           [c,c1],
           [d,d1],
           [e,e1],
           [f,f1],
           [g,g1],
           [h,h1],
           [j,j1],
           [k,k1],
          [m,m1],
          [n,n1]]

In [114]:
results_scerevisiae = results.copy()
results_scerevisiae

[[530, 359],
 [536, 364],
 [549, 371],
 [535, 360],
 [83, 55],
 [41, 24],
 [31, 15],
 [138, 104],
 [50, 41],
 [42, 33],
 [538, 364],
 [538, 361]]

### Simulation time

In [115]:
end_time = timeit.default_timer()
execution_time = end_time - start_time
minutes = int(execution_time // 60)
seconds = int(execution_time % 60)
print("Execution time:", minutes, "minutes and", seconds, "seconds")

Execution time: 94 minutes and 55 seconds


# Examine all results

We combine the saved results for each species into one dataframe for comparison

In [116]:
# simulated cases
cases = ['no restrictions',
          '50% bio',
          '10% bio',
          '90% bio',
          'fluxRangeDiff 0.01',
          'fluxRangeDiff 0.05',
          'fluxRangeDiff 0.10',
          'rxnRef 500',
          'rxnRef 1500',
          'rxnRef 5000',
         'N=100',
         'N=1000']

In [117]:
# dataframe to store the results.
results = pd.DataFrame()
results['cases'] = cases
results['Ecoli'] = results_ecoli
results['putida'] = results_putida
results['Scere'] = results_scerevisiae


The two numbers for each species shown in results represent the number of total reaction targets returned and the number of reactions with an associated gene, from left to right, respectively. 

e.g., **[number of reactions, number of reactions with associated genes]**

The cases refer to the parameter that was chagned, with the setpont specified in the case description. 

In [118]:
results

,cases,Ecoli,putida,Scere
0,no restrictions,"[942, 869]","[939, 758]","[530, 359]"
1,50% bio,"[945, 872]","[940, 762]","[536, 364]"
2,10% bio,"[946, 873]","[940, 757]","[549, 371]"
3,90% bio,"[942, 869]","[941, 763]","[535, 360]"
4,fluxRangeDiff 0.01,"[320, 301]","[364, 308]","[83, 55]"
5,fluxRangeDiff 0.05,"[66, 61]","[72, 66]","[41, 24]"
6,fluxRangeDiff 0.10,"[50, 46]","[49, 44]","[31, 15]"
7,rxnRef 500,"[112, 107]","[150, 140]","[138, 104]"
8,rxnRef 1500,"[56, 54]","[50, 45]","[50, 41]"
9,rxnRef 5000,"[41, 38]","[39, 33]","[42, 33]"


# Analysis of results 

Count and compare the number of reactions with genes that are returned for each case against the number of active reactions with associated genes.

## *E. coli*

In [119]:
# count the number of reactions with genes in E. coli 
count=0
for reac in ecoli.reactions:
    if reac.gene_reaction_rule!="":
        count+=1
count

2130

2130 reactions with an associated gene

In [120]:
# perform FVA to determine how many reactions are active during growth
fva_result = fva(ecoli)


In [121]:
# active reactions
active_reactions = fva_result[(fva_result.minimum != 0) | (fva_result.maximum != 0)]

len(active_reactions)

2054

approximately 2054 reactions are active

In [122]:
# determine how many active genes have an associated gene 
count=0
for reac in active_reactions.index:
    if ecoli.reactions.get_by_id(reac).gene_reaction_rule!="":
        count+=1
count

1735

1735 of the active genes have an associated gene 

### calculating percentages of active reactions with gene ranges returned from FluxRETAP

In [123]:
## highest percent of potentially active reactions returned by FluxRETAP.

878/1735*100
# 878 is the highest number of active reactions returned in the E. coli model
# 1735 is the number of total possible active reactions

50.60518731988473

In [124]:
## lowest percent of potentially active reactions returned by FluxRETAP.

46/1735*100
# 46 is the highest number of active reactions returned in the E. coli model
# 1735 is the number of total possible active reactions

2.6512968299711814

## *P. putida*

In [125]:
results

,cases,Ecoli,putida,Scere
0,no restrictions,"[942, 869]","[939, 758]","[530, 359]"
1,50% bio,"[945, 872]","[940, 762]","[536, 364]"
2,10% bio,"[946, 873]","[940, 757]","[549, 371]"
3,90% bio,"[942, 869]","[941, 763]","[535, 360]"
4,fluxRangeDiff 0.01,"[320, 301]","[364, 308]","[83, 55]"
5,fluxRangeDiff 0.05,"[66, 61]","[72, 66]","[41, 24]"
6,fluxRangeDiff 0.10,"[50, 46]","[49, 44]","[31, 15]"
7,rxnRef 500,"[112, 107]","[150, 140]","[138, 104]"
8,rxnRef 1500,"[56, 54]","[50, 45]","[50, 41]"
9,rxnRef 5000,"[41, 38]","[39, 33]","[42, 33]"


In [126]:
# count the number of reactions with genes 
count=0
for reac in putida.reactions:
    if reac.gene_reaction_rule!="":
        count+=1
count

2060

In [127]:
# perform FVA to determine how many reactions are active during growth
fva_result = fva(putida)


In [128]:
# active reactions
active_reactions = fva_result[(fva_result.minimum != 0) | (fva_result.maximum != 0)]
len(active_reactions)

2058

In [129]:
# determine how many active genes have an associated gene 
count=0
for reac in active_reactions.index:
    if putida.reactions.get_by_id(reac).gene_reaction_rule!="":
        count+=1
count

1582

### calculating percentages of active reactions with gene ranges returned from FluxRETAP

In [130]:
## highest percent of potentially active reactions returned by FluxRETAP

778/1593*100
# 778 is the highest number of active reactions returned in the P. putida model
# 1593 is the number of total possible active reactions

48.83866917765223

In [131]:
## lowest percent of potentially active reactions returned by FluxRETAP

30/1593*100
# 30 is the highest number of active reactions returned in the P. putida model
# 1593 is the number of total possible active reactions

1.8832391713747645

## *S. cerevisiae*

In [132]:
# count the number of reactions with genes 
count=0
for reac in Scerevisiae.reactions:
    if reac.gene_reaction_rule!="":
        count+=1
count

1043

In [133]:
# perform FVA to determine how many reactions are active during growth
fva_result = fva(Scerevisiae)


In [134]:
# active reactions
active_reactions = fva_result[(fva_result.minimum != 0) | (fva_result.maximum != 0)]
len(active_reactions)

1206

In [135]:
# determine how many active genes have an associated gene 
count=0
for reac in active_reactions.index:
    if Scerevisiae.reactions.get_by_id(reac).gene_reaction_rule!="":
        count+=1
count

799

### calculating percentages of active reactions with gene ranges returned from FluxRETAP

In [136]:
## highest percent of potentially active reactions returned by FluxRETAP

368/799*100
# 368 is the highest number of active reactions returned in the S. cerevisiae model
# 799 is the number of total possible active reactions

46.057571964956196

In [137]:
## highest percent of potentially active reactions returned by FluxRETAP

15/799*100
# 15 is the highest number of active reactions returned in the S. cerevisiae model
# 799 is the number of total possible active reactions

1.877346683354193